In [36]:
# 필요한 패키지, 모듈, 라이브러리 추가하는 곳
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

### 데이터 불러와서 (train_images, train_lables), (test_images, test_labes) 형태로 만들기

In [2]:
# 기준 디렉토리 설정
base_dir_train = "C:\\Users\\intel\\Downloads\\trainset(300x300)"

# 디렉토리 목록을 알파벳 순으로 정렬
subdirs_train = sorted([d for d in os.listdir(base_dir_train) if os.path.isdir(os.path.join(base_dir_train, d))])

# 저장할 이미지 리스트들을 위한 빈 딕셔너리 초기화
train_images_dic = {}

# 각 디렉토리 순회
for idx, subdir in enumerate(subdirs_train):
    # 각 디렉토리 내 이미지 파일 목록을 정렬하여 가져옴
    image_files = sorted([f for f in os.listdir(os.path.join(base_dir_train, subdir)) if f.lower().endswith(('.jpg', '.jpeg'))])
    # 각 디렉토리에 해당하는 이미지 리스트 초기화
    train_images_dic[f'train_images_{idx}'] = []

    # 각 이미지 파일을 열고 리스트에 추가
    for image_file in image_files:
        image_path = os.path.join(base_dir_train, subdir, image_file)
        
        try:
            with Image.open(image_path) as img:  # 이미지 열기
                train_images_dic[f'train_images_{idx}'].append(img.copy())  # 이미지 복사하여 리스트에 추가
        except Exception as e:
            print(f"Error opening {image_path}: {e}")

# 이후 train_images_dic을 사용할 수 있음


In [ ]:
# 결과 확인 (예시: train_images_0의 첫 번째 이미지 출력)
train_images_dic['train_images_0'][0]  # train_images_0 리스트의 첫 이미지 출력

In [ ]:
# train_images 딕셔너리에 저장된 각 리스트의 길이를 출력
for key, images in train_images_dic.items():
    print(f"{key}에 있는 이미지 개수: {len(images)}개")

In [ ]:
# 레이블을 저장할 빈 리스트 초기화
train_labels_list = []

# 각 train_images의 개수에 따라 레이블을 추가
for idx, images in enumerate(train_images_dic.values()):
    train_labels_list.extend([idx] * len(images))  # 현재 클래스 인덱스(idx)만큼 반복하여 추가

# 리스트를 NumPy 배열로 변환
train_labels = np.array(train_labels_list, dtype=np.uint8)

# 결과 확인
print(train_labels.shape)
print(train_labels)

In [6]:
# 기준 디렉토리 설정
base_dir_test = "C:\\Users\\intel\\Downloads\\testset(300x300)"

# 디렉토리 목록을 알파벳 순으로 정렬
subdirs_test = sorted([d for d in os.listdir(base_dir_test) if os.path.isdir(os.path.join(base_dir_test, d))])

# 저장할 이미지 리스트들을 위한 빈 딕셔너리 초기화
test_images_dic = {}

# 각 디렉토리 순회
for idx, subdir in enumerate(subdirs_test):
    # 각 디렉토리 내 이미지 파일 목록을 정렬하여 가져옴
    image_files = sorted([f for f in os.listdir(os.path.join(base_dir_test, subdir)) if f.lower().endswith(('.jpg', '.jpeg'))])
    # 각 디렉토리에 해당하는 이미지 리스트 초기화
    test_images_dic[f'test_images_{idx}'] = []

    # 각 이미지 파일을 열고 리스트에 추가
    for image_file in image_files:
        image_path = os.path.join(base_dir_test, subdir, image_file)

        try:
            with Image.open(image_path) as img:
                test_images_dic[f'test_images_{idx}'].append(img.copy())
        except Exception as e:
            print(f'Error opening {image_path}: {e}')

# 이후 test_images_dic을 사용할 수 있음

In [ ]:
# 결과 확인 (예시: test_images_0의 첫 번째 이미지 출력)
test_images_dic['test_images_0'][0]  # test_images_0 리스트의 첫 이미지 출력

In [ ]:
# test_images 딕셔너리에 저장된 각 리스트의 길이를 출력
for key, images in test_images_dic.items():
    print(f"{key}에 있는 이미지 개수: {len(images)}개")

In [ ]:
# 빈 test_labels 리스트 생성
test_labels_list = []

# 각 test_images의 개수에 따라 레이블을 추가
for idx, images in enumerate(test_images_dic.values()):
    test_labels_list.extend([idx] * len(images))  # 현재 클래스 인덱스(idx)만큼 반복하여 추가

# 리스트를 NumPy 배열로 변환
test_labels = np.array(test_labels_list, dtype=np.uint8)

# 결과 확인
print(test_labels.shape)
print(test_labels)

In [10]:
# 기존 train_images 딕셔너리를 기반으로 모든 이미지를 하나의 리스트로 변환
train_images_combined = []

# train_images 딕셔너리의 모든 리스트를 반복하여 추가
for images in train_images_dic.values():
    train_images_combined.extend(images)

# 이제 train_images_combined에는 모든 이미지가 포함됨

In [ ]:
train_images_combined[0]

In [ ]:
# 모든 이미지를 동일한 크기로 리사이즈하여 NumPy 배열로 변환
desired_size = (300, 300)  # 원하는 크기 설정
train_images_resized = []

for image in train_images_combined:
    resized_image = image.resize(desired_size)  # 이미지 리사이즈
    train_images_resized.append(np.array(resized_image))  # NumPy 배열로 변환하여 리스트에 추가

# 리스트를 NumPy 배열로 변환
train_images = np.array(train_images_resized)

# shape 확인
print(train_images.shape)

In [13]:
# 기존 test_images 딕셔너리를 기반으로 모든 이미지를 하나의 리스트로 변환
test_images_combined = []

# test_images 딕셔너리의 모든 리스트를 반복하여 추가
for images in test_images_dic.values():
    test_images_combined.extend(images)

# 이제 train_images_combined에는 모든 이미지가 포함됨

In [ ]:
test_images_combined[0]

In [ ]:
# 모든 이미지를 동일한 크기로 리사이즈하여 NumPy 배열로 변환
test_images_resized = []

for image in test_images_combined:
    resized_image = image.resize(desired_size)  # 이미지 리사이즈
    test_images_resized.append(np.array(resized_image))  # NumPy 배열로 변환하여 리스트에 추가

# 리스트를 NumPy 배열로 변환
test_images = np.array(test_images_resized)

# shape 확인
print(test_images.shape)

### 데이터 랜덤하게 섞기 + 정규화

In [28]:
# 데이터의 인덱스 생성 (train dataset)
indices = np.arange(train_images.shape[0])
np.random.shuffle(indices)  # 인덱스를 랜덤하게 섞기

# 섞인 인덱스를 사용하여 train_images와 train_labels를 재배치
train_images = train_images[indices]
train_labels = train_labels[indices]

In [ ]:
train_labels

In [33]:
# 데이터의 인덱스 생성 (test dataset)
indices2 = np.arange(test_images.shape[0])
np.random.shuffle(indices2)  # 인덱스를 랜덤하게 섞기

# 섞인 인덱스를 사용하여 test_images와 test_labels를 재배치
test_images = test_images[indices2]
test_labels = test_labels[indices2]

In [ ]:
test_labels

In [ ]:
# 이미지 데이터 정규화
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

### CNN 모델 구축하기


In [ ]:
# CNN 모델 생성
model = models.Sequential()

# 첫 번째 합성곱층 + 풀링층
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# 두 번째 합성곱층 + 풀링층
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# 세 번째 합성곱층 + 풀링층
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten층
model.add(layers.Flatten())

# Dense층
model.add(layers.Dense(128, activation='relu'))

# 출력층 (100개의 클래스를 위한 소프트맥스)
model.add(layers.Dense(100, activation='softmax'))

# 모델 요약 출력
model.summary()


In [38]:
# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # 정수 레이블에 적합한 손실 함수
              metrics=['accuracy'])

### 학습 시키기

In [ ]:
# 모델 학습
history = model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2)